In [38]:
class Backtester:
    
    def __init__(self, start_date):
        from tensorflow.keras.models import Sequential
        from tensorflow.keras.layers import Dense
        from tensorflow.keras.models import model_from_json
        from pathlib import Path
        import pandas as pd
        from datetime import datetime
        from datetime import date
        from sklearn.model_selection import train_test_split
        from sklearn.preprocessing import StandardScaler
        import pickle
        
        self.Sequential = Sequential
        self.Dense = Dense
        self.model_from_json = model_from_json
        self.Path = Path
        self.pd = pd
        self.datetime = datetime
        self.tts = train_test_split
        self.StandardScaler = StandardScaler
        self.save_to_path = "../Exported_BuyList/Buy_list"+str(self.datetime.now()).replace(" ", "_").replace(".","_").replace(":","_")+".xlsx"
        self.pickle = pickle
        self.ticker_path = self.Path('../Resources/short_list_for_ML.pkl')
        self.writer = self.pd.ExcelWriter(self.save_to_path, engine="xlsxwriter")
        self.start_date = start_date
        self.model_dict_info = {}
        
        backtest_starter_confirmation = "Backtester has been initalized, please select the paramters for the backtester \n The initialization date used in the model will represent the previous days data \n for which the model will begin to start understanding the characteristics of the stock"
        print(backtest_starter_confirmation)
        
    def load_obj(self, file):
        with open(file, 'rb') as f:
            return self.pickle.load(f)
        
    def get_all_tickers(self):
        tickers = self.load_obj(self.ticker_path)
        return tickers
    
    def return_dataframe(self, symbol, shift=False):
        if shift == False:
            path = self.Path('../FilesExport_DFs_with_TI_pkl/'+symbol+'_data_dict_with_technicals.pkl')
            data_import = self.load_obj(path)
            df = data_import[symbol].copy()
        elif shift == True:
            path = self.Path('../FilesExport_DFs_with_TI_shifted/'+symbol+'_df_with_shifted_TI.pkl')
            data_import = self.load_obj(path)
            df = data_import[symbol].copy()
        return df
    
    def get_no_shift_model(self, symbol):
    
        df = self.return_dataframe(symbol,shift=False)

        ## Set X and y data 
        X = df.drop(columns={'close','adjClose'}).values
        y = df['adjClose'].values

        scaler = self.StandardScaler().fit(X)
        X = scaler.transform(X)

        # load json and create model
        file_path = self.Path('../Model_Data_low_loss/'+symbol+'_model_data.json')
        with open(file_path, "r") as json_file:
            model_json = json_file.read()
        loaded_model = self.model_from_json(model_json)

        # load weights into new model
        file_path = self.Path('../Model_Data_low_loss/'+symbol+'_model_weights.h5')
        loaded_model.load_weights(file_path)

        ## Compile loaded model and print mse score 
        loaded_model.compile(loss="mean_squared_error", optimizer="adam", metrics=["mse"])

        return loaded_model, df , X , y
    
    def get_with_shift_model(self, symbol):
    
        df = self.return_dataframe(symbol,shift=True)

        ## Set X and y data 
        X = df.drop(columns={'close', 'adjClose'}).values
        y = df['adjClose'].values

        scaler = self.StandardScaler().fit(X)
        X = scaler.transform(X)

        # load json and create model
        file_path = self.Path('../Model_Data_shift_low_loss/'+symbol+'_model_data.json')
        with open(file_path, "r") as json_file:
            model_json = json_file.read()
        loaded_model = self.model_from_json(model_json)

        # load weights into new model
        file_path = self.Path('../Model_Data_shift_low_loss/'+symbol+'_model_weights.h5')
        loaded_model.load_weights(file_path)

        ## Compile loaded model and print mse score 
        loaded_model.compile(loss="mean_squared_error", optimizer="adam", metrics=["mse"])

        return loaded_model, df , X , y
    
    
    def evaluate_models(self, symbol):
        
        noshift_model, df , X , y = self.get_no_shift_model(symbol)
        shift_model, df_shift , X_shift , y_shift = self.get_with_shift_model(symbol)
        noshift_score_ = noshift_model.evaluate(X, y, verbose=0)
        noshift_score = noshift_score_[0]
        shift_score_ = shift_model.evaluate(X_shift, y_shift, verbose=0)
        shift_score = shift_score_[0]

        if shift_score < noshift_score:
            self.model_dict_info[symbol] = {
                'model_type' : 'shift',
                'acc' : shift_score
            }
            #dataframe_dict[symbol] = df_shift
        else:
            self.model_dict_info[symbol] = {
                'model_type' : 'no_shift',
                'acc' : noshift_score
            }
      
    def run_universe_evaluation(self):
        universe = self.get_all_tickers()
        
        for ticker in universe:
            self.evaluate_models(ticker)
        

In [35]:
backtest = Backtester('2010-01-01')

Backtester has been initalized, please select the paramters for the backtester 
 The initialization date used in the model will represent the previous days data 
 for which the model will begin to start understanding the characteristics of the stock


In [36]:
dataframe = backtest.return_dataframe('AAA')

In [37]:
dataframe

,open,high,low,close,adjClose,volume,unadjustedVolume,change,changePercent,vwap,...,ShortVolumeNSDQ,ShortExemptVolumeNSDQ,TotalVolumeNSDQ,ShortVolumeNYSE,ShortExemptVolumeNYSE,TotalVolumeNYSE,bollinger_signal,dema_signal,adl_signal,rsi_signal
Date,,,,,,,,,,,,,,,,,,,,,
2016-02-02,10937.799805,11400.000000,10937.799805,11091.900391,11091.900391,844093.0,844093.0,154.10059,1.409,11143.23340,...,0.0,0.0,0.0,0.0,0.0,0.0,0.086565,0.219945,0.769881,0.734666
2016-02-03,11168.900391,11168.900391,10937.799805,10937.799805,10937.799805,647305.0,647305.0,-231.10059,-2.069,11014.83333,...,0.0,0.0,0.0,0.0,0.0,0.0,0.247230,0.211066,0.728040,0.748449
2016-02-04,11168.900391,11168.900391,11014.900391,11014.900391,11014.900391,246134.0,246134.0,-154.00000,-1.379,11066.23372,...,0.0,0.0,0.0,0.0,0.0,0.0,0.165512,0.217213,0.694567,0.579600
2016-02-05,10937.799805,11091.900391,10937.799805,10937.799805,10937.799805,170329.0,170329.0,0.00000,0.000,10989.16667,...,0.0,0.0,0.0,0.0,0.0,0.0,0.183518,0.212432,0.610884,0.498277
2016-02-16,11400.000000,11631.099609,11400.000000,11400.000000,11400.000000,663260.0,663260.0,0.00000,0.000,11477.03320,...,0.0,0.0,0.0,0.0,0.0,0.0,0.613573,0.302596,0.117156,0.501034
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-10-15,24.910000,24.920000,24.900000,24.910000,24.640757,2200.0,2200.0,0.00000,0.000,24.91000,...,120.0,0.0,144.0,0.0,0.0,0.0,0.583102,0.997951,-0.725451,0.683666
2020-10-16,24.906000,24.910000,24.900000,24.910000,24.640757,600.0,600.0,0.00400,0.016,24.90667,...,198.0,0.0,386.0,200.0,0.0,200.0,0.578947,0.996585,-0.856134,0.785665
2020-10-19,24.900000,24.910000,24.900000,24.905001,24.635813,6300.0,6300.0,0.00500,0.020,24.90500,...,199.0,0.0,1049.0,0.0,0.0,0.0,0.584488,0.995219,-0.838743,0.785665
